##### 5.2 Greedy Search Decoding

In [1]:
!pip install torch transformers pandas


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model_name = 'gpt2-xl' # 1.5B
model_name = 'beomi/kykim-gpt3-kor-small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
import pandas as pd

input_txt = "오늘 날씨는"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # 첫 번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # 가장 높은 확률의 토큰을 저장합니다.
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # 예측한 다음 토큰을 입력에 추가합니다.
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,[CLS] 오늘 날씨는 [SEP],오늘 (4.28%),어제 (4.15%),비가 (2.69%),어 (2.45%),내일 (1.98%)
1,[CLS] 오늘 날씨는 [SEP] 오늘,날씨 (7.35%),아침 (5.40%),날씨가 (3.64%),아침에 (3.35%),날씨는 (3.27%)
2,[CLS] 오늘 날씨는 [SEP] 오늘 날씨,정말 (10.65%),참 (4.52%),너무 (3.57%),##를 (3.45%),진짜 (3.21%)
3,[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말,좋네요 (8.93%),춥 (5.02%),덥 (3.39%),좋아요 (2.77%),화 (2.71%)
4,[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말 좋네요,. (35.88%),[SEP] (24.69%),~ (15.09%),! (7.14%),^ (5.94%)
5,[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말 좋네요.,[SEP] (87.90%),. (5.52%),^ (0.72%),ㅎㅎ (0.39%),오늘 (0.32%)
6,[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말 좋네요. [SEP],오늘 (7.58%),어제 (4.79%),오늘은 (2.37%),내일 (2.08%),어 (1.98%)
7,[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말 좋네요. [SEP] 오늘,날씨 (9.00%),아침 (3.86%),날씨가 (3.42%),하루 (3.31%),아침에 (3.18%)


In [4]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 오늘 날씨는 [SEP] 오늘 날씨 정말 좋네요. [SEP]


In [5]:
max_length = 128

input_txt = """재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 이 그림은 한국 화가의 대표작 중 하나다. [SEP]


In [6]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(42000, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=42000, bias=False)
)


##### 5.3 beam search decoding

In [7]:
import numpy as np
sum([np.log(0.5)] * 1024)

-709.7827128933695

In [8]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [9]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\n로그 확률: {logp:.2f}")

[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 이 그림은 한국 화가의 대표작 중 하나다. [SEP]

로그 확률: -19.11


In [10]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 한국화가 김환기 ( 1917 ~ 1994 ) 는 한국전쟁이 한창이던 1950년부터 1955년까지 한국전쟁을 그린 그림들을 많이 그렸다. [SEP]

로그 확률: -48.51


In [11]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 한국화가 김환기 ( 1917 ~ 1994 ) 는 한국전쟁이 한창이던 1950년부터 1955년까지 한국 전쟁에 참전했다. 전쟁이 끝난 직후인 1956년부터 1960년까지 서울과 부산을 오가며 그림을 그렸다. 1961년부터 1971년까지 부산과 대구를 오가면서 그린 그림들은 전쟁과 가난을 그린 한국화 중 가장 유명하다. 전쟁으로 인한 고통과 슬픔을 그

로그 확률: -135.57


##### 5.4 sampling

In [12]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 의 왼손에 파일 625 220 자주 0공원 죄책 한번쯤 침몰자에 티슈쇽 소방대학 강계와 의원 임철훼 결국국장 부 거부폭탄 때문 웃음나무 읽고 개나 밖에서 85장판 타당대한 조작두 가령바트49 샤오 상식 돌아가렴 시작하면 딿 없습니다 튀부터 약물상담 장미 출입 죽기 등이다 ho습 전반적으로소재 방과 토로 전문 부인이 3분의 브랜드에듯이 친근 여자 인적이든 바이올렛종의 이들을 내구성 손목 만들었되고 신데컴게 두려


In [13]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 한국화단의 대표주자로 손꼽히는 김훈 ( 56 ) 은 1970년대 초부터 본격적으로 그림을 그리기 시작했다. [SEP]


##### 5.5 top-k, top-p

In [14]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.5, top_p=0.90)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[CLS] 재난의 시대에는 푸르른 자연을 그린 그림이 유난히 사랑을 받는다. 한 땀 한 땀 세필로 나무와 꽃을 그리고, 해와 달과 별과 작은 새들을 숨겨 놓는 한국 화가가 있다. [SEP] 화가는 자신의 작품을 통해 환경보호와 환경보전, 그리고 지구환경보호를 위해 노력하는 사람들의 모습을 그린다. [SEP]
